In [12]:
import requests

WIKI_API_URL = "https://ja.wikipedia.org/w/api.php"

# 検索キーワードのページ取得
def fetch_search_results(keyword, limit=10):
    params = {
        "action": "query",
        "list": "search",
        "srsearch": keyword,
        "format": "json",
        "srlimit": limit
    }

    headers = {
        "User-Agent": "LYGroup-WikiClient/1.0 (sarimura@lycorp.co.jp)"  # ←識別可能なUA
    }

    try:
        response = requests.get(WIKI_API_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get("query", {}).get("search", [])
    except requests.RequestException as e:
        print("検索リクエストでエラーが発生しました:", e)
        return []

# ページ内容を取得
def fetch_page_content(title):
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True
    }

    headers = {
        "User-Agent": "LYGroup-WikiClient/1.0 (sarimura@lycorp.co.jp)"
    }
    

    try:
        response = requests.get(WIKI_API_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        page = next(iter(data.get("query", {}).get("pages", {}).values()), {})
        return page.get("extract", "ページの内容が見つかりません")
    except requests.RequestException as e:
        print("ページ内容取得でエラーが発生しました:", e)
        return "ページ内容が取得できませんでした。"

In [13]:
# テスト
keyword = "Python"

search_results = fetch_search_results(keyword)
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:3]:
        print(f"- {result['title']}")
    
    # 最初のページタイトル取得
    first_title = search_results[0]['title']
    print("\n【最初のページタイトル】")
    print(first_title)

    # 最初のページ内容
    page_content = fetch_page_content(first_title)
    print("\n【ページ内容】")
    print(page_content[:500])
else:
    print("検索結果が見つかりません。")

【キーワードに該当するページ一覧】
- Python
- モンティ・パイソン
- IronPython

【最初のページタイトル】
Python

【ページ内容】
Python（パイソン）はインタープリタ型の高水準汎用プログラミング言語である。


== 概要 ==
Pythonは1991年にグイド・ヴァン・ロッサムにより開発されたプログラミング言語である。
最初にリリースされたPythonの設計哲学は、ホワイトスペース（オフサイドルール）の顕著な使用によってコードの可読性を重視している。その言語構成とオブジェクト指向のアプローチは、プログラマが小規模なプロジェクトから大規模なプロジェクトまで、明確で論理的なコードを書くのを支援することを目的としている。
Pythonは動的に型付けされていて、ガベージコレクションされている。構造化（特に手続き型）、オブジェクト指向、関数型プログラミングを含む複数のプログラミングパラダイムをサポートしている。Pythonは、その包括的な標準ライブラリのため、しばしば「バッテリーを含む」言語と表現されている。
Pythonのインタプリタは多くのOSに対応している。プログラマーのグローバルコミュニティは、自由かつオープンソース  のリファレンス実装であるCPythonを開発および保守している 。非営利団体であるPyth


In [16]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"

In [17]:
# テスト
# 要約を行うプロンプトを作成
prompt = f"""
以下の文章を要約してください。

# 条件：
- 小学生にもわかるように
- 300文字程度

# 文章：
{page_content[:1000]}
"""

# APIリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

print(response.choices[0].message.content.strip())

Python（パイソン）は、1991年にグイド・ヴァン・ロッサムが作ったプログラミング言語です。Pythonは、コードが読みやすく、わかりやすいことを大切にしています。プログラマーは、小さなプロジェクトから大きなプロジェクトまで、簡単に明確なコードを書くことができます。

Pythonの特徴として、動的に型付けされていて、ガベージコレクションがあること、いろいろなプログラミングのスタイルに対応していることが挙げられます。また、Pythonにはたくさんの便利なツールが用意されていて、必要なものをインターネットからダウンロードして使うことができます。

Pythonは多くのコンピュータで動かせるため、世界中のプログラマーが使っています。Pythonのコミュニティでは、シンプルで簡単なコードを書くことが好まれています。Pythonは「一つの良いやり方があるはず」という考え方を持っていて、使いやすさを大切にしています。


In [8]:
# テスト
# 回答表示
print(response.choices[0].message.content.strip())

Python（パイソン）は、1991年にグイド・ヴァン・ロッサムによって作られたプログラミング言語です。この言語は、コードが読みやすくなるように設計されています。Pythonは、さまざまなプログラミングのスタイルをサポートしていて、簡単に使えるのが特徴です。プログラマーたちは、Pythonを使って小さなプロジェクトから大きなプロジェクトまで、わかりやすいコードを書くことができます。

Pythonは「バッテリーを含む」と呼ばれることもあり、たくさんの便利な機能が最初から用意されています。プログラムを書くとき、必要なツールはインターネットからダウンロードして使うことができます。また、Pythonのコミュニティでは、シンプルで効率的なコードを書くことが大切にされています。Pythonの設計思想には、「物事をするには一つの良い方法がある」という考え方があります。これにより、プログラミングがより楽しく、わかりやすくなっています。


In [20]:
# 類似ワード調査

import time

keyword = "Python"

search_results = fetch_search_results(keyword)
if search_results:
    print("【wikipediaでの類似キーワードページ(最大3件)】")
    for result in search_results[:3]:
        print(f"------ キーワード【{result['title']}】 ------")
    
        """
        # 検索結果の内、最大2件を表示
        first_title = search_results[0]['title']
        print("\n【最初のページタイトル】")
        print(first_title)
        """

        
        # 
        page_content = fetch_page_content(result['title'])
        #print("【ページ内容】")
        #print(page_content[:500])
        

        # 要約を行うプロンプトを作成
        prompt = f"""
        以下の文章を要約してください。

        # 条件：
        - 読者はマーケティング担当者
        - 出力内容の冒頭に「-----wikiページのAIサマリ-----」と出力
        - 箇条書きで3行

        # 文章：
        {page_content[:500]}
        """

        # APIリクエスト
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt},
            ],
            max_tokens=100,
            temperature=0.3
        )
        print(response.choices[0].message.content.strip())

        time.sleep(60) 
else:
    print("検索結果が見つかりません。")

【wikipediaでの類似キーワードページ(最大3件)】
------ キーワード【Python】 ------
-----wikiページのAIサマリ-----
- Pythonは1991年に開発された高水準の汎用プログラミング言語で、可読性を重視した設計哲学を持つ。
- 動的型付けやガベージコレクションを特徴とし、複数のプログラミングパラダイムをサポートしている。
- オープンソースであり、包括
------ キーワード【モンティ・パイソン】 ------
-----wikiページのAIサマリ-----
- モンティ・パイソンは、1969年に始まったBBCのコメディ番組で人気を博したイギリスのコメディグループ。
- 彼らの革新的なスケッチ構成は、アメリカのコメディ番組やポップ・カルチャーに大きな影響を与えた。
- 不条理なスタイル
------ キーワード【IronPython】 ------
-----wikiページのAIサマリ-----  
- IronPythonは、.NET Framework上で動作するPythonの実装で、Python 2.x系と互換性がある。  
- .NETのクラスライブラリをPython文法で利用でき、既存のPythonコード資産も活用可能。  
- IronPythonはC#で実装されており、スクリプト言語としての利用や.NETアセンブ
